In [128]:
%matplotlib inline

# Task 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import  linear_model
import seaborn as sns

import math
np.set_printoptions(precision=3)
plt.rcParams.update({'font.size': 22})
path_X = "../Project2/action.csv"
oX = pd.read_csv(path_X)
path_Y = "../Project2/alert.csv"
oY = pd.read_csv(path_Y)

X = oX.copy()
Y = oY.copy()

In [129]:
attack_names_map = {"Continue":0,"Ping Scan":1,"Install tools":2,"DVWA SQL Injection Exploit":3,"Network service login":4,"Sambacry Explolit":5}
X = X.replace(attack_names_map)
print(X)

      0  1  2  3  4  5  6  7  8  9
0     0  1  5  4  2  1  3  4  2  1
1     0  0  0  1  5  4  2  1  3  4
2     0  0  1  5  4  2  1  3  4  2
3     0  0  0  0  0  0  0  0  1  5
4     0  1  5  4  2  1  3  4  2  1
...  .. .. .. .. .. .. .. .. .. ..
1995  0  1  5  4  2  1  3  4  2  1
1996  0  0  0  1  5  4  2  1  3  4
1997  0  0  0  1  5  4  2  1  3  4
1998  0  0  1  5  4  2  1  3  4  2
1999  0  0  1  5  4  2  1  3  4  2

[2000 rows x 10 columns]


In [130]:
#Divide the alert in k clusters
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


df = pd.DataFrame(columns=['attack','alert',"timestep"])
count = 0
rows_list = []
unique_values = set()
for index, row in X.iterrows():
    
    for col, value in row.items():
        rows_list.append({"attack": value,"alert":Y.iloc[index][col],"timestep":col})
        unique_values.add(Y.iloc[index][col])

best_model = None
best_score = 0
clusters = 0
np_unique = np.array(list(unique_values))

for i in range(6,10):
    kmeans = KMeans(n_clusters = i, random_state = 0, n_init='auto')
    kmeans.fit(np_unique.reshape(-1, 1))
    score = silhouette_score(np_unique.reshape(-1,1), kmeans.labels_, metric='euclidean')
    p_score = '{:.2e}'.format(score)
    print(f"{i}&{p_score}\\\ ")
        
     
    if score > best_score:
        best_model = kmeans
        best_score = score
        clusters = len(kmeans.cluster_centers_)



print(best_score)
print(clusters)

6&6.22e-01\\ 
7&6.04e-01\\ 
8&5.95e-01\\ 
9&5.81e-01\\ 
0.6218135644755878
6


In [7]:
# mapping = {}
# Y = oY.copy()
# pred = best_model.predict(np_unique.reshape(-1,1))
# for i,x in enumerate(np_unique):
#     mapping[x] = pred[i]
# #print(len(mapping))
# Y= Y.replace(mapping)

In [6]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=0)

In [131]:

from scipy.stats import zscore

Y = oY.copy()

tmp = Y.to_numpy().reshape(1,20000)[0].tolist()

mu = np.mean(tmp)
sigma = np.std(tmp)

z_scores = zscore(tmp)
z_scores



array([-0.379,  0.668, -0.502, ..., -0.558, -0.441, -0.557])

In [132]:
best_model = None
best_score = 0
clusters = 0
for i in range(6,10):
    kmeans = KMeans(n_clusters = i, random_state = 0, n_init='auto')
    kmeans.fit(z_scores.reshape(-1, 1))
    score = silhouette_score(z_scores.reshape(-1,1), kmeans.labels_, metric='euclidean')
    p_score = '{:.2e}'.format(score)
    print(f"{i}&{p_score}\\\ ")
        
     
    if score > best_score:
        best_model = kmeans
        best_score = score
        clusters = len(kmeans.cluster_centers_)



print(best_score)
print(clusters)


6&5.72e-01\\ 
7&5.84e-01\\ 
8&5.74e-01\\ 
9&5.47e-01\\ 
0.5837840813088907
7


In [133]:

tmp_z = z_scores.reshape(2000,10)
print(tmp_z)
Z = pd.DataFrame(tmp_z,columns=[str(i) for i in range(10)])


[[-0.379  0.668 -0.502 ... -0.483 -0.499  1.879]
 [-0.775 -0.477 -0.808 ...  2.44  -0.391 -0.516]
 [-0.259 -0.782  1.015 ... -0.631 -0.593 -0.508]
 ...
 [-0.269 -0.502 -0.58  ...  2.228 -0.416 -0.512]
 [-0.602 -0.44   0.678 ... -0.557 -0.573 -0.092]
 [-0.573 -0.503  0.768 ... -0.558 -0.441 -0.557]]


In [134]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense,LSTM
import keras



In [137]:
model = Sequential()
model.add(LSTM(30, input_shape=(10,1)))
#model.add(Dense(30, input_shape=(10,1)))
model.add(Dense(10))
model.compile(loss="mse",optimizer=keras.optimizers.AdamW())

In [138]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Z, test_size=0.30, random_state=0)

In [139]:
history = model.fit(x=y_train, y= X_train,validation_data=(y_test, X_test),epochs=100)


Epoch 1/100
44/44 [==============================] - 3s 20ms/step - loss: 5.9929 - val_loss: 3.9822
Epoch 2/100
44/44 [==============================] - 0s 7ms/step - loss: 2.2480 - val_loss: 1.7758
Epoch 3/100
44/44 [==============================] - 0s 7ms/step - loss: 1.7073 - val_loss: 1.6491
Epoch 4/100
44/44 [==============================] - 0s 7ms/step - loss: 1.6039 - val_loss: 1.5408
Epoch 5/100
44/44 [==============================] - 0s 7ms/step - loss: 1.4660 - val_loss: 1.3567
Epoch 6/100
44/44 [==============================] - 0s 7ms/step - loss: 1.2371 - val_loss: 1.1390
Epoch 7/100
44/44 [==============================] - 0s 8ms/step - loss: 1.0528 - val_loss: 1.0249
Epoch 8/100
44/44 [==============================] - 0s 9ms/step - loss: 0.9522 - val_loss: 0.9443
Epoch 9/100
44/44 [==============================] - 0s 8ms/step - loss: 0.8719 - val_loss: 0.8749
Epoch 10/100
44/44 [==============================] - 0s 8ms/step - loss: 0.8036 - val_loss: 0.8148
Epoch 11

In [140]:
pred = model.predict(y_test,verbose=0)


In [74]:
pred[0]

array([-3.527e-03, -1.077e-02,  1.055e+00,  4.859e+00,  3.990e+00,
        2.045e+00,  9.658e-01,  3.048e+00,  4.077e+00,  2.001e+00],
      dtype=float32)

In [73]:
X_test

,0,1,2,3,4,5,6,7,8,9
405,0,0,1,5,4,2,1,3,4,2
1190,0,1,5,4,2,1,3,4,2,1
1132,0,0,0,0,0,1,5,4,2,1
731,0,0,1,5,4,2,1,3,4,2
1754,0,0,1,5,4,2,1,3,4,2
...,...,...,...,...,...,...,...,...,...,...
841,0,1,5,4,2,1,3,4,2,1
108,0,0,1,5,4,2,1,3,4,2
665,0,0,0,0,1,5,4,2,1,3
1214,0,0,0,1,5,4,2,1,3,4


In [141]:
import math
def approximate_number(number):
    
    if number < 0 :
        return 0
    decimal_part = number - int(number)
    if decimal_part < 0.5:
        # Round down if the decimal is under 0.5
        result = math.floor(number)
    else:
        # Round up if the decimal is 0.5 or greater
        result = math.ceil(number)

    return result

In [142]:
tmp_pred = pred.copy()
tmp_pred = pd.DataFrame(tmp_pred,columns=[str(x) for x in range(10)])
print(tmp_pred)
for idx,(index, row) in enumerate(tmp_pred.iterrows()):
    
        for col, value in row.items():
            tmp_pred.iloc[idx][col] = int(approximate_number(value))


tmp_pred = tmp_pred.astype(int)
print(tmp_pred)


                

            0         1         2         3         4         5         6  \
0   -0.001563 -0.009739  0.981007  4.971360  4.015004  2.045628  0.991152   
1    0.002797  1.000996  4.978779  4.048113  2.050720  1.048233  2.990760   
2   -0.001556  0.023628  0.288092  0.319740  0.311585  1.057374  5.006852   
3   -0.002794 -0.024110  0.952688  4.934631  4.037838  2.067867  1.006589   
4   -0.002465  0.077448  1.088772  4.151167  3.599786  1.809496  0.896695   
..        ...       ...       ...       ...       ...       ...       ...   
595  0.005126  1.006630  5.084984  3.939267  1.944337  0.939252  3.019907   
596 -0.000330 -0.010016  0.992163  4.990530  4.016936  2.031615  0.986985   
597  0.000411 -0.012307 -0.017762  0.002913  0.979899  4.976205  3.994645   
598 -0.011846 -0.055931  0.066611  1.153703  4.885559  3.804556  1.814026   
599 -0.005146 -0.023527  1.017498  5.045705  4.035865  2.006443  1.001480   

            7         8         9  
0    3.014733  4.033745  2.023998  
1  

In [143]:
X_test

tmp_xtest = X_test.copy()

In [144]:
tmp_xtest = tmp_xtest.reset_index()
del tmp_xtest["index"]

In [152]:
def starting_accuracy(pred,seq):
    right = 0
    total = 0
    for idx,e in enumerate(pred.to_numpy()):
        total+=1
        
        
        for k in range(10):
            if seq.iloc[idx].to_numpy().tolist()[k] == 0 and pred.iloc[idx].to_numpy().tolist()[k] == 0:
                continue
            else:
                if seq.iloc[idx].to_numpy().tolist()[k] != 0 and  pred.iloc[idx].to_numpy().tolist()[k] != 0:
                    right+=1
                    break
                else:
                    break
    return right/total

acc_start = starting_accuracy(tmp_pred,tmp_xtest)
acc_start

0.9766666666666667

In [146]:
def overall_accuracy(pred,seq):
    right = 0
    total = 0
    for idx,e in enumerate(pred.to_numpy()):
        
        for k in range(10):
            total+=1
            if pred.iloc[idx].to_numpy().tolist()[k] == seq.iloc[idx].to_numpy().tolist()[k]:
                right+=1
    return right/total

acc_ov = overall_accuracy(tmp_pred,tmp_xtest)
acc_ov

0.9928333333333333

In [109]:
#OVERFITTING CHECK


overfittingX = pd.read_csv("../data2-Testing/action.csv")
attack_names_map = {"Continue":0,"Ping Scan":1,"Install tools":2,"DVWA SQL Injection Exploit":3,"Network service login":4,"Sambacry Explolit":5}
ovX = overfittingX.replace(attack_names_map)
overfittingY = pd.read_csv("../data2-Testing/alert.csv")
ovtmp = Y.to_numpy().reshape(1,20000)[0].tolist()
ovz_scores = zscore(ovtmp)
ovz_scores
ovz = ovz_scores.reshape(2000,10)
oZ = pd.DataFrame(ovz,columns=[str(i) for i in range(10)])
print(oZ)
pred = model.predict(oZ,verbose=0)
print(pred)
pred = pd.DataFrame(pred,columns=[str(x) for x in range(10)])

for idx,(index, row) in enumerate(pred.iterrows()):
    
        for col, value in row.items():
            pred.iloc[idx][col] = int(approximate_number(value))


pred = pred.astype(int)

acc_start = starting_accuracy(pred,X)
acc_ov = overall_accuracy(pred,X)

print(acc_start,acc_ov)



             0         1         2         3         4         5         6  \
0    -0.379093  0.668370 -0.502380 -0.515193  0.018571  2.048226 -0.488118   
1    -0.774822 -0.476514 -0.808182  1.100361 -0.625910 -0.737111 -0.558948   
2    -0.258947 -0.781833  1.014785 -0.008020 -0.511566 -0.580463  1.886018   
3    -0.565716 -0.713420 -0.599077 -0.734451 -0.620833 -0.516160 -0.664347   
4    -0.413178  0.460473 -0.474580 -0.471196 -0.195128  2.116397 -0.635580   
...        ...       ...       ...       ...       ...       ...       ...   
1995 -0.506973  0.798185  0.007209 -0.596176 -0.144121  2.591901 -0.347425   
1996 -0.578287 -0.390696 -0.571276  1.253625 -0.366039 -0.394322 -0.065797   
1997 -0.268859 -0.501655 -0.579737  0.830095 -0.367489 -0.482074 -0.423331   
1998 -0.602219 -0.439528  0.678040 -0.029294 -0.449923  0.095445  2.387389   
1999 -0.573210 -0.503347  0.767726 -0.639206 -0.626635 -0.488359  2.412288   

             7         8         9  
0    -0.482558 -0.498996  